# 파이썬으로 사용하는 시맨틱 커널(Semantic Kernel in Python)
[시맨틱 커널](https://github.com/microsoft/semantic-kernel)(SK)은 마이크로소프트가 공개한 오픈소스로, 대규모 언어 모델(LLM)을 애플리케이션에 빠르고 간단하게 탑재할 수 있는 SDK다. 시맨틱 커널을 사용하면 기존 프로그래밍 언어로 최신 LLM 모델과 프롬프트를 쉽게 다룰 수 있으며 템플릿화, 체인화, 내장 메모리, 플래닝 등의 기능을 제공한다.

이 예제 코드는 시맨틱 커널 공식 [Samples](https://github.com/microsoft/semantic-kernel/tree/main/python/samples)에 기반하여 작성한 것이다. Microsoft Learn에 있는 [공식문서](https://learn.microsoft.com/semantic-kernel/overview/)도 참고 바란다.

## 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 GPT-4o 모델
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI GPT-4o 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.12.4로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 패키지 설치

In [ ]:
!pip install semantic-kernel==1.1.2

## Azure OpenAI 설정
연동을 위해 필요한 정보는 보안을 위해 하드코딩 하지 말고 .env 파일에서 불러오는 것을 권장한다.  
- [.env.sample](.env.sample) 파일 이름을 `.env`로 변경하고 필요한 값을 설정한다.

### Azure OpenAI 사용시
아래 정보가 필요합니다.
```
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
```
### OpenAI 사용시
아래 정보가 필요합니다.
```
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
```

In [ ]:
#AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
#AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
#AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = "gpt-4o"
#AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002"

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding, AzureChatCompletion, AzureTextEmbedding

kernel = Kernel()

# Azure OpenAI Service 사용 여부
useAzureOpenAI = True

service_id = None
# 시맨틱 커널에서 사용할 OpenAI 서비스 설정
if useAzureOpenAI:
    service_id = "azure-openai-chat"
    azure_chat_service = AzureChatCompletion(service_id=service_id)
    #azure_text_embedding = AzureTextEmbedding(service_id=service_id)
    kernel.add_service(azure_chat_service)
    #kernel.add_service(azure_text_embedding)
else:
    # OpenAI 사용시
    service_id = "openai-chat"
    oai_text_service = OpenAIChatCompletion(service_id=service_id)
    kernel.add_service(oai_text_service)

## 요약
프롬프트를 사용해서 콘텐츠를 요약하는 시맨틱 함수를 만들어보자. 이 함수는 요약할 텍스트를 입력받는다.

In [ ]:
prompt = """
다음 설명을 6살인 유치원생도 이해할 수 있게 요약해줘.
유치원생이 모르는 단어는 쉬운 단어로 변환해줘.

{{$input}}
"""

from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if useAzureOpenAI:
    summarize_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=400,
        temperature=0.0,
        top_p=0.5,
    )
else:
    # OpenAI 사용시
    summarize_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=400,
        temperature=0.0,
        top_p=0.5,
    )
    
summarize_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=summarize_settings,
)

summarize_function = kernel.add_function(
    plugin_name="summarizePlugin",
    function_name="summarizeFunction",
    prompt_template_config=summarize_config,
)

In [ ]:
input_text = """
중성자별은 초신성 폭발 직후 무거운 별이 중력붕괴하여 만들어진 밀집성의 일종이다.
중성자별은 현재까지 관측된 우주의 천체 중 블랙홀 다음으로 밀도가 크다. 거의 12 ~ 13 km의 반지름에 태양의 두 배에 달하는 무거운 질량을 가지고 있다.
중성자별은 거의 대부분이 순전하가 없고 양성자보다 약간 더 무거운 핵자인 중성자로 구성되어 있다. 
이들은 양자 축퇴압에 의해 붕괴되지 않고 유지되는데 이는 매우 뜨거우며 두 개의 중성자(또는 페르미 입자)가 동시에 같은 위치 및 양자 상태를 취할 수 없다는 원리인 파울리 배타 원리를 통해 설명되는 현상이다.
중성자별의 질량은 최소 1.1 태양질량에서 3 태양질량(M☉)까지이다. 관측된 것 중 가장 무거운 것은 2.01 M☉이다. 
중성자별의 표면온도는 보통 ~6×105 K이다. 중성자별의 전체 밀도는 3.7×1017에서 5.9×1017 kg/m3 (태양의 밀도의 2.6×1014 ~ 4.1×1014 배)이다.
"""

In [ ]:
from semantic_kernel.functions import KernelArguments

summary = await kernel.invoke(
    function=summarize_function, 
    arguments=KernelArguments(input=input_text)
)

print(summary)

## 채팅 이력을 활용하는 챗봇

In [ ]:
sk_prompt = """
너는 한국의 무신정권 역사에 관한 문제를 답변해주는 역사 교수야.
명확한 출처를 찾을 수 없거나, 답변할 수 없는 질문에는 '모르겠습니다'라고만 답변해줘.

{{$history}}
User: {{$user_input}}
ChatBot: """

### 시맨틱 함수로 등록하기

In [ ]:
if useAzureOpenAI:
    chat_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.5,
    )
else:
    # OpenAI 사용시
    chat_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.5,
    )
    
chat_config = PromptTemplateConfig(
    template=sk_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=chat_settings,
)

chat_function = kernel.add_function(
    plugin_name="chatPlugin",
    function_name="chatFunction",
    prompt_template_config=chat_config,
)

### Chat history

ChatHistory는 채팅 이력을 유지하는 데 사용된다.

In [ ]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()

### 채팅 시작하기

In [ ]:
arguments = KernelArguments(user_input="최우는 어떤 인물이었나요?", history=chat_history)
bot_answer = await kernel.invoke(
    function=chat_function, 
    arguments=arguments
)

print(bot_answer)

### 출력값으로 채팅 이력 업데이트하기

In [ ]:
chat_history.add_user_message(arguments['user_input'])
chat_history.add_assistant_message(str(bot_answer))
print(chat_history.messages)

### 연속적인 채팅이 가능하도록 함수 작성하기

In [ ]:
async def chat(input_text: str) -> None:
    print(f"User: {input_text}")

    # 사용자가 보낸 메시지를 처리하고 응답을 획득한다.
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # 응답을 표시한다.
    print(f"ChatBot: {answer}")

    # 채팅 이력에 새로 주고 받은 대화를 추가한다.
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [ ]:
await chat("최우가 간행에 기여한 경전의 이름은?")

In [ ]:
await chat("최우는 팔만대장경의 간행에 기여했습니다.")

In [ ]:
await chat("팔만대장경은 어떤 책인가요?")

In [ ]:
await chat("팔만대장경은 고려 시대에 제작된 불교 경전의 목판 인쇄본입니다.")

채팅 후에는 `chat_history`에 쌓인 전체 채팅 이력을 확인할 수 있습니다.

In [ ]:
print(chat_history.messages)

## 플러그인 사용하기
파일에서 플러그인과 함수를 불러온다. `./samples/skills/FunSkill` 디렉토리의 구조는 플러그인 구조와 동일하다.

https://learn.microsoft.com/semantic-kernel/agents/plugins/?tabs=python

In [ ]:
# note: samples 폴더에 있는 플러그인 사용하기
skills_directory = "./samples/skills"

fun_plugins = kernel.add_plugin(plugin_name="FunSkill", parent_directory=skills_directory)
joke_function = fun_plugins["Joke"]

불러온 함수는 다음과 같이 실행할 수 있다.

In [ ]:
joke_arguments = KernelArguments(input="선비에 대하여", style="한국의 웃음 코드에 맞게")

result = await kernel.invoke(
    function=joke_function, 
    arguments=joke_arguments
)
print(result)

In [ ]:
# 스트리밍 실행시
response = kernel.invoke_stream(function=joke_function, arguments=joke_arguments)
async for message in response:
    print(str(message[0]), end="")

## Sequential 플래너
`SequentialPlanner`는 제공된 지시나 질문을 해결하기 위해 단계별 Plan을 생성한다.


### 계산기 플러그인 불러오기
플래너는 사용할 수 있는 플러그인을 알아야 한다. 여기서는 라이브러리에 미리 정의된 `MathPlugin`을 불러온다.

In [ ]:
from semantic_kernel.core_plugins.math_plugin import MathPlugin

kernel.add_plugin(MathPlugin(), "math")

In [ ]:
from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel=kernel, service_id=service_id)

In [ ]:
ask = "202와 990의 합계는 몇이지?"

sequential_plan = await planner.create_plan(goal=ask)

print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

`SequentialPlanner`가 질문을 받아 문제를 해결할 방법을 설명하고 있다.
위 실행 계획에서 알 수 있듯이 AI는 사용자의 요구를 충족시키기 위해 어떤 함수를 호출해야 하는지를 결정한다.

### 플랜 실행
완성된 실행 계획을 실제로 실행해보자.

In [ ]:
results = await sequential_plan.invoke(kernel)
print(results)

## 플러그인 조합하기
### 플래너에 플러그인 제공하기
플래너는 사용할 수 있는 플러그인을 알아야 한다. 여기서는 `./samples/skills` 폴더에 정의된 `SummarizeSkill`과 `WriterSkill`을 사용할 수 있도록 불러오고 있다. 여기에는 많은 시맨틱 함수가 포함되어 있으며 플래너는 그 중 일부를 지능적으로 선택한다.

네이티브 함수를 포함시킬 수도 있다. 여기서는 `TextSkill`을 추가한다.

In [ ]:
from semantic_kernel.core_plugins.text_plugin import TextPlugin

skills_directory = "./samples/skills/"
summarize_skill = kernel.add_plugin(plugin_name="SummarizeSkill", parent_directory=skills_directory)
writer_skill = kernel.add_plugin(plugin_name="WriterSkill", parent_directory=skills_directory)
text_skill = kernel.add_plugin(TextPlugin(), "TextPlugin")

In [ ]:
ask = """내일은 발렌타인데이라 몇 가지 데이트 아이디어가 필요해.
여자친구가 셰익스피어를 좋아하니까 셰익스피어 스타일로 작성해줘.
프랑스어로 작성하고, 문자는 대문자로 변경해줘.
"""

sequential_plan = await planner.create_plan(goal=ask)
print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

인라인 플러그인도 정의해서 플래너가 사용할 수 있게 만든다. 반드시 함수 이름과 플러그인 이름을 붙여야 한다.

In [ ]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
rewrite_config = PromptTemplateConfig(
    template=sk_prompt,
    name="rewrite",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True)
    ],
    execution_settings=chat_settings,
)
shakespeare_function = kernel.add_function(
    function_name="shakespeare",
    plugin_name="ShakespeareSkill",
    prompt_template_config=rewrite_config
)

새 플랜을 생성한다.

In [ ]:
new_plan = await planner.create_plan(goal=ask)
print("The plan's steps are:")
for step in new_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

새로 생성한 플랜을 실행한다.

In [ ]:
results = await new_plan.invoke(kernel)
print(results)